# GitHub Repository 기반 Q&A 챗봇


In [ ]:
from dotenv import load_dotenv

load_dotenv()

LangChain 추적


In [ ]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Code Analysis"

### 데이터 준비


GitHub Repository 를 다운로드 받습니다. 이번 예제는 langchain-ai 공식 Repository 로 진행합니다.

- !git clone 을 사용하여 Repository clone

다른 저장소의 파일을 사용하고 싶다면, `root_dir`을 여러분의 저장소의 루트 디렉토리로 변경하세요.


저는 `/Users/teddy/Dev/github/langchain` 위치에 `langchain` reposotory 를 clone 하였습니다. 아래의 경로는 본인의 경로에 맞게 바꾸어 주어야 합니다.


In [ ]:
!ls "/Users/teddy/Dev/github/langchain/libs"

## 도큐먼트 로더


여기서 모든 패키지의 파일을 불러오지 않습니다. 핵심 기능을 포함하는 특정 폴더의 파일만 불러오도록 아래와 같이 정의해 주었습니다.


In [3]:
# Root 경로
repo_root = "/Users/teddy/Dev/github/langchain/libs"

# 불러오고자 하는 패키지 경로
repo_core = repo_root + "/core/langchain_core"
repo_community = repo_root + "/community/langchain_community"
repo_experimental = repo_root + "/experimental/langchain_experimental"
repo_parters = repo_root + "/partners"
repo_text_splitter = repo_root + "/text_splitters/langchain_text_splitters"
repo_cookbook = repo_root + "/cookbook"

In [4]:
# langchain의 여러 모듈을 가져옵니다.
from langchain_text_splitters import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser

# 불러온 문서를 저장할 빈 리스트를 생성합니다.
py_documents = []

for path in [repo_core, repo_community, repo_experimental, repo_parters, repo_cookbook]:
    # GenericLoader를 사용하여 파일 시스템에서 문서를 로드합니다.
    loader = GenericLoader.from_filesystem(
        path,  # 문서를 불러올 경로
        glob="**/*",  # 모든 하위 폴더와 파일을 대상으로 함
        suffixes=[".py"],  # .py 확장자를 가진 파일만 대상으로 함
        parser=LanguageParser(
            language=Language.PYTHON, parser_threshold=30
        ),  # 파이썬 언어의 문서를 파싱하기 위한 설정
    )
    # 로더를 통해 불러온 문서들을 documents 리스트에 추가합니다.
    py_documents.extend(loader.load())

print(f".py 파일의 개수: {len(py_documents)}")

.py 파일의 개수: 4938


다음은 `.mdx` 확장자를 가진 파일을 `TextLoader` 를 사용하여 불러옵니다. `.mdx` 파일은 Jupyter Notebook 파일을 마크다운 형식으로 변환한 파일이며, 유용한 예제를 포함하고 있으므로 이를 DB 에 추가하기 위해 도큐먼트 형식으로 로드압니다.


In [7]:
import os

# TextLoader 모듈을 불러옵니다.
from langchain_community.document_loaders import TextLoader

# 검색할 최상위 디렉토리 경로를 정의합니다.
root_dir = "/Users/teddy/Dev/github/langchain/"

mdx_documents = []
# os.walk를 사용하여 root_dir부터 시작하는 모든 디렉토리를 순회합니다.
for dirpath, dirnames, filenames in os.walk(root_dir):
    # 각 디렉토리에서 파일 목록을 확인합니다.
    for file in filenames:
        # 파일 확장자가 .mdx인지 확인하고, 경로 내 '*venv/' 문자열이 포함되지 않는지도 체크합니다.
        if (file.endswith(".mdx")) and "*venv/" not in dirpath:
            try:
                # TextLoader를 사용하여 파일의 전체 경로를 지정하고 문서를 로드합니다.
                loader = TextLoader(os.path.join(dirpath, file), encoding="utf-8")
                # 로드한 문서를 분할하여 documents 리스트에 추가합니다.
                mdx_documents.extend(loader.load())
            except Exception:
                # 파일 로드 중 오류가 발생하면 이를 무시하고 계속 진행합니다.
                pass

# 최종적으로 불러온 문서의 개수를 출력합니다.
print(f".mdx 파일의 개수: {len(mdx_documents)}")

.mdx 파일의 개수: 308


## Chunk 분할


파일들을 청크로 나누어 봅시다.


In [8]:
# RecursiveCharacterTextSplitter 모듈을 가져옵니다.
from langchain_text_splitters import RecursiveCharacterTextSplitter

# RecursiveCharacterTextSplitter 객체를 생성합니다. 이 때, 파이썬 코드를 대상으로 하며,
# 청크 크기는 2000, 청크간 겹치는 부분은 200 문자로 설정합니다.
py_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=2000, chunk_overlap=200
)

# py_docs 변수에 저장된 문서들을 위에서 설정한 청크 크기와 겹치는 부분을 고려하여 분할합니다.
py_docs = py_splitter.split_documents(py_documents)

# 분할된 텍스트의 개수를 출력합니다.
print(f"분할된 .py 파일의 개수: {len(py_docs)}")

mdx_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000, chunk_overlap=200)

# mdx_docs 변수에 저장된 문서들을 위에서 설정한 청크 크기와 겹치는 부분을 고려하여 분할합니다.
mdx_docs = mdx_splitter.split_documents(mdx_documents)

# 분할된 텍스트의 개수를 출력합니다.
print(f"분할된 .mdx 파일의 개수: {len(mdx_docs)}")

분할된 .py 파일의 개수: 8549
분할된 .mdx 파일의 개수: 722


로드한 도큐먼트를 합칩니다.


In [9]:
combined_documents = py_docs + mdx_docs
print(f"총 도큐먼트 개수: {len(combined_documents)}")

총 도큐먼트 개수: 9271


## Embedding


In [10]:
# langchain_openai와 langchain의 필요한 모듈들을 가져옵니다.
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore

# 로컬 파일 저장소를 사용하기 위해 LocalFileStore 인스턴스를 생성합니다.
# './cache/' 디렉토리에 데이터를 저장합니다.
store = LocalFileStore("./cache/")

# OpenAI 임베딩 모델 인스턴스를 생성합니다. 모델명으로 "text-embedding-3-small"을 사용합니다.
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small", disallowed_special=())

# CacheBackedEmbeddings를 사용하여 임베딩 계산 결과를 캐시합니다.
# 이렇게 하면 임베딩을 여러 번 계산할 필요 없이 한 번 계산된 값을 재사용할 수 있습니다.
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, store, namespace=embeddings.model
)

## Vector DB


- `from_documents`: 이 메서드는 문서 컬렉션과 이에 해당하는 임베딩을 받아 벡터 인덱스를 생성합니다. 여기서는 `combined_documents` 로부터 벡터를 생성하고, 이 벡터들은 `cached_embeddings` 을 통해 임베딩된 데이터를 사용합니다.
- `save_local`: 이 메서드는 생성된 FAISS 인덱스를 지정된 로컬 폴더에 저장합니다. 이 폴더명은 `FAISS_DB_INDEX` 변수에 저장되어 있습니다.


In [11]:
# langchain_community 모듈에서 FAISS 클래스를 가져옵니다.
from langchain_community.vectorstores import FAISS

# 로컬에 저장할 FAISS 인덱스의 폴더 이름을 지정합니다.
FAISS_DB_INDEX = "langchain_faiss"

# combined_documents 문서들과 cached_embeddings 임베딩을 사용하여
# FAISS 데이터베이스 인스턴스를 생성합니다.
db = FAISS.from_documents(combined_documents, cached_embeddings)

# 생성된 데이터베이스 인스턴스를 지정한 폴더에 로컬로 저장합니다.
db.save_local(folder_path=FAISS_DB_INDEX)

저장한 FAISS 데이터베이스를 불러옵니다. 이후 실행시에는 새롭게 DB 에 저장할 필요없이 아래 코드만 실행하면 됩니다.


In [12]:
# langchain_community 모듈에서 FAISS 클래스를 가져옵니다.
from langchain_community.vectorstores import FAISS

# FAISS 클래스의 load_local 메서드를 사용하여 저장된 벡터 인덱스를 로드합니다.
db = FAISS.load_local(
    FAISS_DB_INDEX,  # 로드할 FAISS 인덱스의 디렉토리 이름
    cached_embeddings,  # 임베딩 정보를 제공
    allow_dangerous_deserialization=True,  # 역직렬화를 허용하는 옵션
)

## Retriever


In [13]:
# MMR을 사용하여 검색을 수행하는 retriever를 생성합니다.
faiss_retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 10})

In [14]:
# langchain.retrievers 모듈에서 BM25Retriever 클래스를 가져옵니다.
from langchain.retrievers import BM25Retriever

# 문서 컬렉션을 사용하여 BM25 검색 모델 인스턴스를 생성합니다.
bm25_retriever = BM25Retriever.from_documents(
    combined_documents  # 초기화에 사용할 문서 컬렉션
)

# BM25Retriever 인스턴스의 k 속성을 10으로 설정하여,
# 검색 시 최대 10개의 결과를 반환하도록 합니다.
bm25_retriever.k = 10

In [15]:
# langchain.retrievers 모듈에서 EnsembleRetriever 클래스를 가져옵니다.
from langchain.retrievers import EnsembleRetriever

# EnsembleRetriever 인스턴스를 생성합니다.
# 이때, BM25 검색 모델과 FAISS 검색 모델을 결합하여 사용합니다.
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever],  # 사용할 검색 모델의 리스트
    weights=[0.6, 0.4],  # 각 검색 모델의 결과에 적용할 가중치
    search_type="mmr",  # 검색 결과의 다양성을 증진시키는 MMR 방식을 사용
)

## 파이프라인 연결

chain 을 구성합니다.


### 프롬프트


In [17]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """당신은 20년차 AI 개발자입니다. 당신의 임무는 주어진 질문에 대하여 최대한 문서의 정보를 활용하여 답변하는 것입니다.
문서는 Python 코드에 대한 정보를 담고 있습니다. 따라서, 답변을 작성할 때에는 Python 코드에 대한 상세한 code snippet을 포함하여 작성해주세요.
최대한 자세하게 답변하고, 한글로 답변해 주세요. 주어진 문서에서 답변을 찾을 수 없는 경우, "문서에 답변이 없습니다."라고 답변해 주세요.
답변은 출처(source)를 반드시 표기해 주세요.

#참고문서:
{context}

#질문:
{question}

#답변: 

출처:
- source1
- source2
- ...                             
"""
)

### LLM 정의


In [18]:
from langchain.callbacks.base import BaseCallbackHandler
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.callbacks.manager import CallbackManager
from langchain_core.runnables import ConfigurableField
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from langchain_community.chat_models import ChatOllama
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic


class StreamCallback(BaseCallbackHandler):
    def on_llm_new_token(self, token: str, **kwargs):
        print(token, end="", flush=True)


llm = ChatOpenAI(
    model="gpt-4-turbo-preview",
    temperature=0,
    streaming=True,
    callbacks=[StreamCallback()],
).configurable_alternatives(
    # 이 필드에 id를 부여합니다.
    # 최종 실행 가능한 객체를 구성할 때, 이 id를 사용하여 이 필드를 구성할 수 있습니다.
    ConfigurableField(id="llm"),
    # 기본 키를 설정합니다.
    default_key="gpt4",
    claude=ChatAnthropic(
        model="claude-3-opus-20240229",
        temperature=0,
        streaming=True,
        callbacks=[StreamCallback()],
    ),
    gpt3=ChatOpenAI(
        model="gpt-3.5-turbo",
        temperature=0,
        streaming=True,
        callbacks=[StreamCallback()],
    ),
    ollama=ChatOllama(
        model="EEVE-Korean-10.8B:long",
        callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    ),
)

In [19]:
# 체인을 생성합니다.
rag_chain = (
    {"context": ensemble_retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

## 질의-응답 테스트


In [20]:
answer = rag_chain.with_config(configurable={"llm": "gpt4"}).invoke(
    "PromptTemplate 사용방법을 알려주세요"
)

`PromptTemplate` 사용 방법에 대해 설명드리겠습니다. `PromptTemplate`은 사용자 입력을 받아서 완전히 형식화된 프롬프트로 변환하는 데 도움을 주는 도구입니다. 이는 사용자가 모델에 직접 지시사항을 제공하지 않고도, 원하는 출력을 얻기 위한 입력을 쉽게 형식화할 수 있게 해줍니다.

기본적인 사용 방법은 다음과 같습니다:

1. `PromptTemplate` 클래스를 임포트합니다.
2. `PromptTemplate`의 인스턴스를 생성하면서, 템플릿 문자열을 전달합니다.
3. `.format()` 메서드를 사용하여 템플릿에 변수를 채워 넣습니다.

예를 들어, 어떤 회사나 제품에 대한 설명만 제공하고, 모델이 그에 맞는 회사 이름을 생성하도록 하고 싶은 경우 다음과 같이 할 수 있습니다:

```python
from langchain.prompts import PromptTemplate

# 템플릿 생성
prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}?")

# 템플릿에 변수 채우기
formatted_prompt = prompt.format(product="colorful socks")

print(formatted_prompt)
```

출력:
```
What is a good name for a company that makes colorful socks?
```

`PromptTemplate`은 또한 메시지 목록을 생성하는 데에도 사용될 수 있습니다. 이 경우, 프롬프트는 내용뿐만 아니라 각 메시지(역할, 목록에서의 위치 등)에 대한 정보도 포함합니다. 가장 흔한 경우는 `ChatPromptTemplate`이 `ChatMessageTemplates`의 목록으로 구성되어 있습니다. 각 `ChatMessageTemplate`은 해당 `ChatMessage`를 형식화하는 방법(역할, 내용 등)에 대한 지시사항을 포함합니다

In [21]:
answer = rag_chain.with_config(configurable={"llm": "ollama"}).invoke(
    "PromptTemplate 사용방법을 알려주세요"
)

Python의 Langchain 라이브러리는 자연어 처리(NLP) 작업에 사용할 수 있는 다양한 도구와 유틸리티를 제공합니다. PromptTemplate은 사용자가 모델에게 입력할 메시지를 생성하는 데 도움을 주는 클래스입니다. 이 클래스는 문자열 형식을 사용하여 사용자 정의 가능한 템플릿을 만들 수 있게 해줍니다.

PromptTemplate을 사용하려면 먼저 Langchain 라이브러리를 설치해야 합니다:
```bash
pip install langchain
```
설치 후, 다음 코드를 사용하여 PromptTemplate 인스턴스를 생성할 수 있습니다:
```python
from langchain import PromptTemplate

# 템플릿 정의
template = "What is the capital of {country}?"

# PromptTemplate 인스턴스 생성
prompt_template = PromptTemplate(template=template)

# 변수 추가
input_variables = {"country": "France"}
partial_prompt = prompt_template.partial(**input_variables)

# 완성된 템플릿 출력
print(partial_prompt)
```
이 예제에서 `"What is the capital of {country}?"`라는 문자열 형식을 가진 템플릿을 정의했습니다. 그런 다음 `PromptTemplate` 클래스를 사용하여 이 템플릿으로 인스턴스를 생성하고, `partial()` 메서드를 사용하여 변수(예: "France")를 추가하여 완성된 템플릿을 만들었습니다. 마지막으로 `print()` 함수를 사용하여 출력물을 표시했습니다.

Langchain 라이브러리는 또한 PromptTemplate과 함께 사용할 수 있는 여러 유틸리티와 클래스를 제공합니다. 예를 들어, `BasePromptTemplate` 클래스는 사용자가 모델에게 입력할 메시지를 생성하는 데 도

In [22]:
answer = rag_chain.with_config(configurable={"llm": "claude"}).invoke(
    "PromptTemplate 사용방법을 알려주세요"
)

PromptTemplate을 사용하는 방법은 다음과 같습니다:

1. PromptTemplate 임포트하기
```python
from langchain.prompts import PromptTemplate
```

2. PromptTemplate 인스턴스 생성하기
- template 인자에 프롬프트 템플릿 문자열을 전달합니다. 
- 템플릿에서 변수는 중괄호({})로 표시합니다.
```python
prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}?") 
```

3. PromptTemplate 포맷팅하기
- format() 메소드를 호출하여 프롬프트 템플릿의 변수에 값을 채웁니다.
```python
prompt.format(product="colorful socks")
```

4. (선택사항) 프롬프트 템플릿 파셜링하기 
- 프롬프트 템플릿의 일부 변수만 지정하고 싶다면 partial() 메소드를 사용할 수 있습니다.
  
5. (선택사항) 프롬프트 템플릿 합성하기
- 여러 개의 프롬프트 템플릿을 합쳐 하나의 프롬프트를 만들 수 있습니다.

PromptTemplate을 사용하면 사용자 입력값을 모델에 그대로 전달하는 대신, 입력값을 적절한 프롬프트 형식으로 가공할 수 있습니다. 이는 언어 모델을 활용한 애플리케이션 개발 시 유용하게 활용될 수 있습니다.

출처:
- /Users/teddy/Dev/github/langchain/docs/docs/modules/model_io/index.mdx
- /Users/teddy/Dev/github/langchain/libs/core/langchain_core/prompts/string.py

In [23]:
answer = rag_chain.invoke("WebbaseLoader 사용법에 대해 알려주세요")

`WebBaseLoader`는 HTML 페이지를 로드하고 `BeautifulSoup`을 사용하여 파싱하는 클래스입니다. 이 로더는 `urllib`을 사용하여 웹 페이지를 로드하고, `BeautifulSoup`을 사용하여 HTML을 파싱합니다. 사용자는 웹 페이지의 경로, 헤더 템플릿, SSL 검증 여부, 프록시 설정, 실패 시 계속 진행 여부, 인코딩 자동 설정, 인코딩 지정, 요청당 초당 요청 수, 기본 파서, 요청에 대한 추가 인자, 상태 코드에 대한 예외 발생 여부, `BeautifulSoup`의 `get_text` 메소드와 관련된 인자, 그리고 `BeautifulSoup` 생성자에 전달할 추가 인자를 설정할 수 있습니다.

다음은 `WebBaseLoader`의 기본 사용 예시입니다:

```python
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()
```

이 예시에서는 `bs4.SoupStrainer`를 사용하여 특정 HTML 클래스를 가진 요소만 파싱하도록 설정하고 있습니다. 이를 통해 전체 HTML 문서 대신 원하는 부분만 추출할 수 있습니다. `WebBaseLoader`의 인스턴스를 생성할 때 `web_paths` 매개변수에 로드하고자 하는 웹 페이지의 URL을 지정합니다. `bs_kwargs` 매개변수에는

In [ ]:
answer = rag_chain.invoke(
    "retriever 의 get_relevant_documents 기능은 어떻게 사용하나요?"
)

In [ ]:
answer = rag_chain.invoke("Agent 에서 TavilySearch 도구를 사용하는 예시를 보여주세요.")